# Thème Image - TP2 - Utilisation de l’histogramme pour la correction et le traitement d’images

* Vérifiez que vous vous trouvez bien dans le dossier de travail pour ce TP
* Si ça n'est pas le cas, pour y accèder utilisez les commandes `pwd` (print working directory), `cd` (change directory) et `ls`(list contents) pour naviguer sur le disque.
* Changez de repertoire jusqu'a voir tous les fichiers necessaires au TP.
* Importez les modules dont vous aurez besoin en exécutant la cellule suivante (`Ctrl+Enter`).
* Familiarisez vous avec les fonctions mises à votre disposition dans le fichier `utils.py`; elles vous seront utiles pour la suite!

In [1]:
ls

_TP0_TEST_FigMatplotlib.png  TP1_Correction.ipynb  TP3_Correction.ipynb
TP0_Tuto.ipynb               TP2_Correction.ipynb


In [1]:
import numpy as np
import matplotlib.pyplot as plt
from copy import copy
from utils import *

test_utils()
plt.ion()

NameError: name 'test_utils' is not defined

# Histogramme et histogramme cumulé
## Histogramme

Python possède des instructions (dans la librairie `numpy`; voir le fichier `utils.py`) permettant de calculer automatiquement l'histogramme d'une image. Utiliser ces fonctions revient à utiliser une "boîte noire" dont on ne connaît pas le fonctionnement interne.

Nous utiliserons des boîtes noires plus tard afin de gagner du temps (et parce que leur implémentation est souvent très optimisée), mais nous allons aussi re-coder certains algorithmes (en "boîtes blanches") afin de bien en comprendre le fonctionnement.

Nous pourrons alors comparer les résultats produits par une boîte noire à ceux produits par notre boîte blanche, afin de vérifier que notre implémentation est correcte.

# Exercice 1
## Tracés d'histogrammes

Dans le cas des images, nous voulons compter le nombre de pixels dont la valeur est $p$, $\forall p \in \lbrace 0,\ldots,255\rbrace$.

Affichez les images suivantes et tracez leurs histogrammes :

`papillon.bmp`, `fruits.bmp`, `manoir.bmp`, `desert.bmp`

Commentez l'aspect général des images et l'allure de leurs
histogrammes.

In [ ]:
# Charger et afficher l'image
img = load_and_show_image('./fruits.bmp')

# Show histogram: black box
bbox_hist = show_histogram(img, title='Histogramme - black box')
show_histogram(img, cumulative=False, title='Histogramme CUMULE - black box')

# Compute histogram from scratch
hist = np.zeros(256)
for i in range(img.shape[0]):
    for j in range(img.shape[1]):
        hist[img[i,j]] += 1
        
# Show histogram: white box
show_bars(hist, title='Histogramme - white box')

# Comparer les resultats black box et white box
print(f'Difference entre black box et white box=\n{bbox_hist - hist}')

# Exercice 2
## Effet d'une transformation sur l'histogramme
On considère les fonctions de transformation suivantes :
* $f_1(p) = p + 50 $
* $f_3(p) = 0.5(p-127) + 127 $
* $f_5(p) = 2(p-127) + 127 $

Modifez le code ci-dessous pour appliquer ces transformations à quelques images.
    
Affichez les images transformées et leurs histogrammes, et commentez la différence par rapport à l'original.

Dans quels cas y a-t-il perte d'information par saturation des valeurs à 0 ou 255 ?

Que se passe-t-il si on évite le bloc "clip" en entrant clip = False ?

In [ ]:
def f1(p):
    return p+50

def f2(p):
    return 0.5 * (p - 127) + 127

def f5_3(p):
    return 2 * (p - 127) + 127

# Charger et afficher l'image
img = load_and_show_image('./manoir.bmp')

# Afficher l'histogramme
show_histogram(img, title='Histogramme de Fruits')

# Appliquer la transformation
img_2 = copy(img)
clip = True
for i in range(img_2.shape[0]):
    for j in range(img_2.shape[1]):
        # A COMPLETER : CHOISIR LA FONCTION A APPLIQUER ICI
        new_val = f5_3(img[i,j])
        # ------ Que se passe-t-il si on désactive (clip = False au dessus) ce bloc de code ?
        # Quel problème avec les valeurs des pixels dans les images
        # observe-t-on ?
        if clip:
            if new_val > 255:
                img_2[i, j] = 255
            elif new_val < 0:
                img_2[i, j] = 0
            else:
              img_2[i, j] = new_val
        else:
            img_2[i,j] = new_val

# Afficher l'image
show_image(img_2, title='Image Fruits transformee')

# Afficher l'histogramme
_ = show_histogram(img_2, title='Histogramme de image Fruits transformee')

# Exercice 3
## Histogramme cumulé

Etant donné l'histogramme $h(p)$ d'une image, l'histogramme cumulé
peut être défini de deux manières.
1. Par une formule explicite
$(\star) H(p) = \sum_{q = 0}^{p}h(q), \ \forall p \in\lbrace 0,\ldots, 255\rbrace.$

2. Par récurrence : $H(0) = h(0)$ et pour $p\in{1, \ldots, 255}, H(p) = H(p-1) + h(p)$.

En pratique, l'histogramme cumulé $H$ est stocké sous la forme d'un tableau
à 256 entrées, comme l'histogramme $h$ que vous avez déjà calculé, il faut faire attention aux indices !

Completez la cellule ci-dessous pour qu'elle réalise le calcul de l'histogramme cumulé, puis testez votre fonction.

Quelle est la valeur du dernier élément du tableau $H$ ? A quoi correspond-elle ?

In [ ]:
# Charger et afficher l'image
#img = load_and_show_image('./papillon.bmp', title='Papillon')
img = load_and_show_image('./tomates.bmp', title='Tomates')

# Calculer l'histogramme: white box (on en aura besoin plus bas)
hist = np.zeros(256)
for i in range(img.shape[0]):
    for j in range(img.shape[1]):
        hist[img[i, j]] += 1

# Afficher l'histogramme
show_bars(hist, title='Histogramme de Papillon')

# Histogramme cumule: calcul lent (méthode 1, lente)
print_timestamp('hist LENT: start')
H_1 = np.zeros(256)
for p in range(256):
    H_1[p] = 0
    for pp in range(p):
        H_1[p] += hist[pp] 
print_timestamp('hist LENT: end')

# Histogramme cumule: calcul optimise (méthode 2, optimisée)
print_timestamp('hist RAPIDE: start')
H_2 = np.zeros(256)
H_2[0] = hist[0]
for p in range(1,256):
    H_2[p] += H_2[p-1] + hist[p]
print_timestamp('hist RAPIDE: end')

# Afficher l'histogramme cumule
show_bars(H_1, title='Histogramme CUMULE de Papillon')

# Exercice 4
## Histogramme et correction de contraste

Dans cette partie, nous allons voir comment l'histogramme permet de choisir la transformation affine à appliquer à une image donnée. La transformation, dite LUT (ou Look Up Table) sera affichée par la cellule ci-dessous.
    
L'idée est d'identifier l'intervalle $[p_0,p_1]$ de valeurs dans lequel se trouvent la majorité des pixels de l'image puis de répartir ces valeurs sur l'intervalle  $[0,255]$.

Tracez l'histogramme de l'image `desert.bmp`.

* Quel est le plus petit intervalle qui contient tous les pixels ?
* Quelle est la transformation affine qui permet de répartir ces valeurs sur l'intervalle $[0,255]$ ?
* Modifiez les paramètres dans le code de la cellule ci-dessous puis appliquez la LUT 'idéale' puis quelques autres transformations à l'image.
* Commentez les différences au niveau des images et des histogrammes.

In [ ]:
import numpy as np

# Verifier que la fonction LUT est bien definie
LUT_x = np.arange(256)
LUT_y = np.array([LUT(x, 100, 200) for x in LUT_x])
show_function(LUT_x, LUT_y, title='LUT')

# Charger et afficher l'image
img = load_and_show_image('./desert.bmp', title='Desert')

# Afficher l'histogramme
show_histogram(img, title='Histogramme de Desert')

# Appliquer une LUT a l'image
img_2 = copy(img)

# Transformation sans perte d'information
for i in range(img.shape[0]):
    for j in range(img.shape[1]):
        img_2[i,j] = LUT(img[i,j],75,230)

# Transformation avec perte d'information
for i in range(img.shape[0]):
    for j in range(img.shape[1]):
        img_2[i,j] = LUT(img[i,j],75,230)

# Afficher la nouvelle image
show_image(img_2, title='LUT(Desert)')

# Afficher le nouvel histogramme
_ = show_histogram(img_2, title='Histogramme de LUT(Desert)')


# Exercice 5
## Détermination automatique de $p_0$ et $p_1$

Soit $I$ une image de taille $M\times N$.

Soit $s\in[0,100]$ une valeur choisie par l'utilisateur. On choisit $p_0$ (respectivement $p_1$) de telle manière que $s\%$ des pixels de l'image $I$ aient une valeur plus petite que $p_0$ (respectivement plus grande que $p_1$).

Autrement dit, soit $H$ l'histogramme cumulé de $I$, $p_0$ est la plus petite valeur $p$ telle que $\frac{H(p)}{MN} \geq s/100$, et $p_1$ est la plus grande valeur $p$ telle que $\frac{H(p)}{MN} \leq 1 - s/100$.

* Modifiez la cellule ci-dessous pour déterminer $p_0$ et $p_1$.

_**Astuce** : pour $p_0$, parcourir l'histogramme cumulé $H$ à partir de la valeur de pixel 0 ; pour $p_1$, parcourir $H$ dans le sens inverse (donc à partir de l'indice 255)._

* Testez votre code avec l'image `desert.bmp`, par exemple avec un seuil de 1% :

* Les valeurs de $p_0$ et $p_1$ sont-elles proches de celles que vous avez choisies dans l'exercice précédent ? Essayez avec d'autres valeurs du seuil et affichez les images correspondantes.

* Faites de même pour les images `manoir.bmp` et `fruits.bmp`.


In [ ]:
# Charger et afficher l'image
img = load_and_show_image('./papillon.bmp', title='Papillon')
#img = load_and_show_image('./tomates.bmp', title='Tomates')

# Calculer l'histogramme: white box (on en aura besoin plus bas)
hist = np.zeros(256)
for i in range(img.shape[0]):
    for j in range(img.shape[1]):
        hist[img[i, j]] = hist[img[i, j]] + 1

# Afficher l'histogramme
show_bars(hist, title='Histogramme de Papillon', cumulative=True)
show_histogram(img, cumulative = True)

# Histogramme cumule: méthode 2 (optimisée)
H = np.zeros(256)
H[0] = hist[0]
for i in range(1, 256):
    H[i] = H[i-1] + hist[i]

# Choisir un pourcentage s
s = 5  # (en pourcents (%))

# Trouver p0
nb_pixels = img.shape[0] * img.shape[1]
p0 = 0
while H[p0] / nb_pixels < s/100:
    p0 += 1
p0 += 1

# Trouver p1
p1 = 255
while H[p1] / nb_pixels > 1 - s / 100:
    p1 -= 1
p1 -= 1
# A COMPLETER


# Autre facon de le coder
# for p in range(0, 256):
#     if H[p]/nb_pixels >= s/100:
#         p0 = p
#         break
# for p in range(255, -1, -1):
#     if H[p]/nb_pixels <= 1 - s/100:
#         p1 = p
#         break

# Valeurs
print(f'p0= {p0}')
print(f'p1= {p1}')

# Verifier que la fonction LUT est bien definie
LUT_x = np.arange(256)
# A COMPLETER
LUT_y = np.array([LUT(x, p0, p1) for x in LUT_x])
show_function(LUT_x, LUT_y, title='LUT')

# Appliquer une LUT a l'image
img_2 = copy(img)
for i in range(256):
    for j in range(256):
        img_2[i,j] = LUT(img[i,j], p0, p1)
# Afficher la nouvelle image
show_image(img_2, title='LUT(Papillon)')

# Afficher le nouvel histogramme
_ = show_histogram(img_2, title='Histogramme de LUT(Papillon)')

# Exercice 6
## Egalisation d'histogramme

Pour une image $I_1(u,v)$ de taille $M\times N$ dont l'histogramme cumulé est $H_1(p)$, l'égalisation d'histogramme consiste à transformer $I_1(u,v)$ en une image $I_2(u,v) = T(I_1(u,v))$ de manière à ce que le nouvel histogramme cumulé $H_2$ soit le plus linéaire possible. Pour cela, on applique la
transformation suivante aux pixels de l'image :

$ T(p) = \frac{255}{MN}H_1(p). $

* Dans la cellule ci-dessous, définir une fonction `egalisation` qui calcule la nouvelle valeur `T(p)` puis mettre en oeuvre l'égalisation d'histogramme et la tester sur `fruits.bmp`.

* Commenter les allures de l'histogramme et de l'histogramme cumulé de la nouvelle image par rapport à ceux de l'image originale.

* Commenter l'aspect visuel de la nouvelle image. Que pensez-vous du résultat par rapport à la correction affine ?

* Appliquer une deuxième fois l'égalisation d'histogramme. Qu'observez-vous ? Comment l'expliquer ?

* Que pensez-vous de l'égalisation appliquée à l'image `cellules.bmp` ?


In [ ]:
# Charger et afficher l'image
img = load_and_show_image('./papillon.bmp')
#img = load_and_show_image('./cellules.bmp')

# Calculer l'histogramme: white box (on en aura besoin plus bas)
hist = np.zeros(256)
for i in range(img.shape[0]):
    for j in range(img.shape[1]):
        hist[img[i, j]] = hist[img[i, j]] + 1

# Afficher l'histogramme
show_bars(hist, title='Histogramme')

# Histogramme cumule: calcul optimise
H1 = np.zeros(256)
H1[0] = hist[0]
for i in range(1, 256):
    H1[i] = H1[i-1] + hist[i]

# Afficher histogramme cumule
show_bars(H1, title='Histogramme CUMULE')

# NB: vous pouvez remplacer les 2 calculs d'hist plus haut par des appels
# à la fonction show_histogram (voir la doc de cette fonction dans utils.py)

# Transformation d'egalisation d'histogramme
def egalisation(p):
    MN = img.size
    return H1[p] * 255 / MN
    
# Appliquer l'egalisation d'histogramme
img_2 = copy(img)
for i in range(img_2.shape[0]):
    for j in range(img_2.shape[1]):
        img_2[i,j] = egalisation(img[i,j])
        
# Afficher la nouvelle image
show_image(img_2, title='Image apres egalisation')

# Calculer l'histogramme egalise
show_histogram(img_2, cumulative=False, title='Histogramme apres egalisation')

# Calculer l'histogramme cumulé egalise
_ = show_histogram(img_2, cumulative=True, title='Histogramme CUMULE apres \
egalisation')

# Exercice 7
## Segmentation par seuillage

La segmentation d'une image consiste à identifier des classes de pixels suivant certains critères.

Par exemple dans l'image `tomates.bmp`, on constate que le fond est nettement plus sombre que les tomates, et on peut choisir comme critère un seuil d'intensité $s$ :
un pixel donné appartient à la classe **fond** si sa valeur est inférieure à $s$, et à la classe **tomate** sinon.

* Visualiser l'histogramme de l'image `tomates.bmp` et identifier une valeur plausible du seuil $s$ qui permette de séparer le fond des tomates.

* Dans la cellule ci-dessous implémenter la fonction `threshold` suivante :

$ T(p) =
\left\lbrace
\begin{array}{ccc}
      255 & \text{si} & p > s,\\
      0 & \text{sinon.}
\end{array}
\right.
$  
Vous pouvez la tester de la façon suivante, en faisant varier le seuil $s$ autour de la valeur que vous avez évaluée précédemment.

* Dans le même esprit que le TP précédent, vous pouvez créer un masque puis utiliser le produit entre deux `numpy.ndarray` pour afficher les tomates sans le fond, ou le fond sans les tomates :

* Il y a un message caché dans le fond de l'image `tomates.bmp` saurez-vous le rendre clairement visible ?

In [ ]:
# Charger et afficher l'image
img = load_and_show_image('./tomates.bmp', title='Tomates')

# Afficher l'histogramme
show_histogram(img, title='Histogramme de Tomates')

# Fonction de seuillage
def threshold(img, s):
    img_th = copy(img)
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            if img[i,j] > s:
                img_th[i,j] = 255
            else:
                img_th[i,j] = 0
    return img_th

# Appliquer un seuillage
# A COMPLETER
img_2 = threshold(img, 50)
show_image(img_2, title='Seuillage(Tomates)')

# Afficher les tomates sans le fonds.
# A COMPLETER
img_3 = img * (img_2 / 255)
        
show_image(img_3, title='Tomates sans le fonds')

# Afficher le fonds sans les tomates
# A COMPLETER
img_4 = img - img_3
show_image(img_4, title='Le fonds sans les tomates')

# A COMPLETER : Chercher le message caché dans le fonds.


In [ ]:
plt.figure();plt.imshow(img_4, vmin=0, vmax=30, cmap='gray')
h = np.zeros(256,dtype=int)
for p in img_4.flatten():
    h[int(p)] += 1
H = np.zeros(256)
H[0] = h[0]
for i in range(1,256):
    H[i] = h[int(i)] + H[i-1]
MN = img_4.flatten().shape[0]
lut = H * 255 / MN

egal_4 = np.zeros_like(img_4)
for i in range(egal_4.shape[0]):
    for j in range(egal_4.shape[1]):
        egal_4[i,j] = lut[int(img_4[i,j])]

show_image(egal_4, title='Le fonds sans les tomates')

_ = show_histogram(egal_4, cumulative=True, title='Histogramme CUMULE apres \
egalisation')

